# Feature Engineering

---

Introduction

Feature engineering is the heart of predictive analytics, transforming raw data into meaningful features that enhance the model’s performance. In our logistics optimization project, this stage is essential for creating features that capture the nuances of our data and improve our predictive capabilities. By building on insights from the EDA, we’ll derive new features like Cost per Mile and Delivery Efficiency (measured as Distance per Time) to quantify efficiency. These features provide refined lenses through which we can analyze operational costs, helping to differentiate efficient routes and drivers from less efficient ones.

In addition to creating new features, we will standardize and encode categorical data to prepare the dataset for machine learning. This structured dataset, rich with engineered features, will form the basis for our predictive models, ultimately allowing us to make informed, data-driven recommendations for logistics optimization.

---

Conclusion

Our feature engineering efforts have enriched the dataset, creating a series of well-defined predictors that will be invaluable in the modeling stage. Cost per Mile and Delivery Efficiency provide quantifiable measures of efficiency, while standardized and encoded categorical data ensure that our models can interpret the dataset effectively. These engineered features not only capture essential operational characteristics but also serve as levers for optimization. By completing this stage, we have crafted a dataset that is primed for predictive modeling, ready to unlock actionable insights into logistics operations.

---


Loading and Preparing Data

In [129]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [130]:
import sys
import os

# 'scripts' is in the same parent directory as 'notebooks'
sys.path.append(os.path.abspath("../scripts"))

from feature_engineering import create_features

In [131]:
# Load the cleaned data
logistics_df = pd.read_csv('../data/raw/logistics_df.csv')
logistics_df.head()

,Route ID,Driver ID,Delivery Time (hours),Date,Fuel Costs (USD),Delivery Start Time,Distance Traveled (miles),Estimated Distance (miles),Weather Conditions,Traffic Conditions,...,Cost per Gallon (USD),Total Fuel Cost (USD),Insurance Costs (USD),Breakdown Repair Costs (USD),Overtime Labor Costs (USD),Fuel Surcharge (USD),Idle Cost (USD),Total Operational Cost (USD),Fuel Cost per Mile,Delivery Efficiency Score
0,3a2d8f81-6054-45fa-b13a-24cd540ee76d,75149212-0e83-4e5b-83d4-3bd0c53f3030,6.880603,2023-02-09,225.206331,22:34:14,571.318744,2339.271800,Clear,Moderate,...,3.0,305.072583,53.293350,0.0,0.000000,15.806772,0.368712,340.228658,0.394187,0.015323
1,e8e92a39-62e1-4743-baee-c8544fbe59ee,9f47ca00-1ae7-4f76-878a-9a283d2c2ec2,4.967601,2023-06-17,570.840791,03:38:06,1593.917947,1468.769307,Heavy Rain,Heavy,...,3.5,992.973038,60.448295,0.0,0.000000,40.066147,1.623336,684.194856,0.358137,0.047800
2,f6b4a887-f0cc-4528-a178-583c6c4d1fc4,fc05b7ca-0b9f-427d-9874-e21be1c6e521,8.094009,2023-12-18,447.387571,09:01:13,1127.075475,2270.634894,Clear,Severe,...,3.0,601.835368,36.241719,0.0,2.384313,31.401218,0.855647,559.776989,0.396946,0.007551
3,257e5a3f-78d3-48a1-8584-f508f2ee3670,4dd64322-0cc6-44a8-a2b7-b5f2663c9cae,7.297488,2023-04-22,886.741516,14:26:53,1607.293364,2476.720021,Heavy Rain,Light,...,3.0,858.261948,75.561809,0.0,0.000000,62.238573,0.330756,1052.419293,0.551699,0.008019
4,c74ae987-3974-490b-9707-bb325937916f,4ad93d48-9ad6-46a7-933c-57f8603a7f03,7.797437,2023-07-07,871.855361,03:17:26,379.177698,965.593024,Heavy Rain,Light,...,3.5,236.218704,26.602416,0.0,0.000000,61.193744,2.769288,1008.848800,2.299332,0.013692


In [132]:
# Create Cost per Mile
logistics_df['Cost per Mile'] = logistics_df['Fuel Costs (USD)'] / logistics_df['Distance Traveled (miles)']


In [133]:
logistics_df['Cost per Ton-Mile'] = logistics_df['Total Operational Cost (USD)'] / (logistics_df['Load Weight (tons)'] * logistics_df['Distance Traveled (miles)'])

In [134]:
# Create Delivery Efficiency feature
logistics_df['Delivery Efficiency'] = logistics_df['Distance Traveled (miles)'] / logistics_df['Delivery Time (hours)']


In [135]:
# Map Customer Satisfaction to numeric values (assuming an ordinal scale)
satisfaction_map = {
    'Very Dissatisfied': 1,
    'Dissatisfied': 2,
    'Neutral': 3,
    'Satisfied': 4,
    'Very Satisfied': 5
}

# Apply the mapping
logistics_df['Customer Satisfaction'] = logistics_df['Customer Satisfaction'].map(satisfaction_map)

inefficient_routes = logistics_df[
    (logistics_df['Fuel Cost per Mile'] > logistics_df['Fuel Cost per Mile'].quantile(0.75)) |
    (logistics_df['Delays (hours)'] > logistics_df['Delays (hours)'].quantile(0.75)) |
    (logistics_df['Customer Satisfaction'] < 3)]

Traffic Conditions:
        
Purpose: Helps analyze how traffic impacts delivery time and fuel consumption, allows recommending optimal delivery times or routes.


In [136]:
 # Categorize routes by length
conditions = [
    (logistics_df['Distance Traveled (miles)'] < 100),
    (logistics_df['Distance Traveled (miles)'] >= 100) & (logistics_df['Distance Traveled (miles)'] < 300),
    (logistics_df['Distance Traveled (miles)'] >= 300)
    ]
categories = ['Short', 'Medium', 'Long']
logistics_df['Route Length'] = np.select(conditions, categories)

    # Map weather conditions to severity levels
weather_map = {
    'Sunny': 'Low',
    'Cloudy': 'Low',
    'Windy': 'Moderate',
    'Rainy': 'High',
    'Snowy': 'High',
    'Stormy': 'Severe'
    }
logistics_df['Weather Severity'] = logistics_df['Weather Conditions'].map(weather_map)


In [137]:
weather_map = {'Sunny': 1, 'Cloudy': 2, 'Windy': 3, 'Rainy': 4, 'Snowy': 5, 'Stormy': 6}
logistics_df['Weather Severity Score'] = logistics_df['Weather Conditions'].map(weather_map)

In [138]:
logistics_df['Severe Weather'] = logistics_df['Weather Conditions'].apply(lambda x: 1 if x in ['Rainy', 'Snowy', 'Stormy'] else 0)

In [139]:
severity_mapping = {'Clear': 0, 'Light Rain': 1, 'Heavy Rain': 2, 'Snow': 3, 'Fog': 2}  # Example severity scores
logistics_df['Weather Severity Index'] = logistics_df['Weather Conditions'].map(severity_mapping)

In [140]:
inefficient_routes = logistics_df[(logistics_df['Fuel Cost per Mile'] > logistics_df['Fuel Cost per Mile'].quantile(0.75)) |
                               (logistics_df['Delays (hours)'] > logistics_df['Delays (hours)'].quantile(0.75)) |
                               (logistics_df['Customer Satisfaction'] < 3)]

In [141]:
route_delivery_time = logistics_df.groupby('Route ID')['Delivery Time (hours)'].mean()

In [142]:
logistics_df['Fuel Cost per Mile'] = logistics_df['Fuel Costs (USD)'] / logistics_df['Distance Traveled (miles)']

In [143]:
logistics_df['Performance Score'] = (1 / (1 + logistics_df['Delays (hours)']) * logistics_df['Customer Satisfaction'] * (1 / (1 + logistics_df['Fuel Cost per Mile'])))

In [144]:
logistics_df['Driver Consistency Score'] = logistics_df.groupby('Driver ID')['Delays (hours)'].transform('mean')

In [145]:
# Calculate Fuel Cost per Mile
logistics_df['Fuel Cost per Mile'] = logistics_df['Fuel Costs (USD)'] / logistics_df['Distance Traveled (miles)']


In [146]:
logistics_df = pd.get_dummies(logistics_df, columns=['Traffic Conditions'], drop_first=False)

In [147]:
# Add a binary target variable for On-Time Delivery
logistics_df['On-Time Delivery'] = np.where(logistics_df['Delays (hours)'] > 1, 0, 1)  # 1 = On-Time, 0 = Delayed

New Features for Driver Experience and Weather Severity
a) Driver Experience on Specific Routes

Assign numeric values based on driver experience to analyze its impact on long and challenging routes.

In [148]:
# Map driver experience to numeric values
experience_map = {'Junior': 1, 'Intermediate': 2, 'Senior': 3}
logistics_df['Driver Experience Level'] = logistics_df['Driver Experience'].map(experience_map)

# Interaction term: experienced drivers on long routes
logistics_df['Experience on Long Routes'] = logistics_df['Driver Experience Level'] * (logistics_df['Route Length'] == 'Long').astype(int)

In [149]:
sns.histplot(logistics_df['Cost per Mile'], bins=30, kde=True)
plt.title("Distribution of Cost per Mile")
plt.show()


In [150]:
# Save the engineered data
logistics_df.to_csv('../data/processed/engineered_data.csv', index=False)



Enhanced Modeling
a) Improved Feature Engineering for Regression Model

Add interaction terms and apply hyperparameter tuning for better performance.

In [151]:
# Interaction term: Distance and Truck Condition
logistics_df['Distance * Truck Condition'] = logistics_df['Distance Traveled (miles)'] * logistics_df['Truck Condition']

# Define features and target
X = logistics_df[['Distance Traveled (miles)', 'Truck Condition', 'Driver Ratings', 'Load Weight (tons)', 
                  'Traffic Conditions_Severe', 'Weather Severity Index', 'Distance * Truck Condition']]
y = logistics_df['Total Operational Cost (USD)']

# Hyperparameter tuning with GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X, y)

print("Best Parameters:", grid_search.best_params_)
print("Best RMSE:", np.sqrt(-grid_search.best_score_))

Best Parameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 200}
Best RMSE: 330.031825747286
